# Read this first 

* each comment and text line added here is important and relevant , dont skim through just the code
* if the comment asks you to revisit some concepts/modules , do so
* this is not complete code on its own , you need to fill in your own code to eventually start building a model

In [191]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import numpy as np

In [192]:
datafile_train="../input/edvancer-project-4-manf/Hazard_train.csv"
datafile_test="../input/edvancer-project-4-manf/Hazard_test_share.csv"
bd_train=pd.read_csv(datafile_train)
bd_test=pd.read_csv(datafile_test)

In [193]:
bd_train.head()

,Id,Hazard,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,...,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
0,83642,2,10,23,4,N,K,Y,B,D,...,2,37,1,11,2,Y,N,E,2,1
1,92530,6,3,18,2,B,H,N,B,B,...,2,34,1,12,7,Y,N,C,3,3
2,86131,4,5,20,4,B,K,N,B,B,...,1,40,1,17,4,Y,N,D,1,1
3,88666,1,4,7,5,N,H,N,B,B,...,2,37,1,11,6,Y,N,D,2,1
4,11122,21,10,21,6,N,C,Y,B,C,...,2,40,1,6,6,N,N,D,5,5


In [194]:
bd_train.shape

(40799, 34)

In [195]:
cat_cols=bd_train.select_dtypes(['object']).columns

In [196]:
cat_cols

Index(['T1_V4', 'T1_V5', 'T1_V6', 'T1_V7', 'T1_V8', 'T1_V9', 'T1_V11',
       'T1_V12', 'T1_V15', 'T1_V16', 'T1_V17', 'T2_V3', 'T2_V5', 'T2_V11',
       'T2_V12', 'T2_V13'],
      dtype='object')

In [197]:
bd_train['data']='train'
bd_test['data']='test'
all_data=pd.concat([bd_train,bd_test],0,sort=False)

In [198]:
all_data.shape

(50999, 35)

In [199]:
for col in cat_cols:
    k=all_data[col].value_counts()
    cats=k[k>=100].index[:-1]
    
    for cat in cats:
        name=col+'_'+cat
        all_data[name]=(all_data[col]==cat).astype(int)
        
    del all_data[col]

In [200]:
all_data.shape

(50999, 94)

In [201]:
all_data.head()

,Id,Hazard,T1_V1,T1_V2,T1_V3,T1_V10,T1_V13,T1_V14,T2_V1,T2_V2,...,T2_V5_A,T2_V5_B,T2_V5_C,T2_V5_D,T2_V11_Y,T2_V12_N,T2_V13_E,T2_V13_A,T2_V13_C,T2_V13_D
0,83642,2.0,10,23,4,2,10,0,52,14,...,0,0,1,0,1,1,1,0,0,0
1,92530,6.0,3,18,2,8,10,3,39,14,...,1,0,0,0,1,1,0,0,1,0
2,86131,4.0,5,20,4,7,15,1,66,10,...,0,0,1,0,1,1,0,0,0,1
3,88666,1.0,4,7,5,8,15,3,48,11,...,0,1,0,0,1,1,0,0,0,1
4,11122,21.0,10,21,6,12,10,1,42,11,...,1,0,0,0,0,1,0,0,0,1


In [202]:
X=all_data.drop(['Id','Hazard','data'],1)[all_data['data']=='train']
y=all_data['Hazard'][all_data['data']=='train']

In [203]:
X.shape, y.shape

((40799, 91), (40799,))

In [204]:
x_test=all_data.drop(['Id','Hazard','data'],1)[all_data['data']=='test']

In [205]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()

X = mms.fit_transform(X)

x_test = mms.transform(x_test)

In [206]:
from sklearn.model_selection import train_test_split

x_train,x_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=45)

In [207]:
xgb = XGBRegressor(objective = 'count:poisson',
                    eta = 0.05,
                    n_estimators = 100,
                    min_child_weight = 5,
                    max_depth = 8,
                    gamma =2,
                    subsample= 0.7,
                    colsample_bytree= 0.8,
                    colsample_bylevel=0.8,
                    reg_lambda=0, 
                    reg_alpha=10,
                    eval_metric = 'poisson-nloglik',
                    seed = 0
                   )

In [208]:
xgb.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=0.8, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False, eta=0.05,
             eval_metric='poisson-nloglik', gamma=2, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.0500000007,
             max_bin=256, max_cat_to_onehot=4, max_delta_step=0.699999988,
             max_depth=8, max_leaves=0, min_child_weight=5, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, objective='count:poisson', predictor='auto',
             random_state=0, ...)

In [209]:
predi = xgb.predict(x_val)

from sklearn.metrics import mean_absolute_error as mae

score = 1-(mae(y_val,predi)/5.4)

score

0.513035522136748

In [210]:
submissions=pd.DataFrame({'Hazard':xgb.predict(x_test)})

In [211]:
submissions.to_csv('Project4_submission1_yogeswarareddy.csv',index=False)

In [212]:
sadf

NameError: name 'sadf' is not defined

from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

#xgb_params = {'reg_lambda':[i/10 for i in range(0,50)],
#              'reg_alpha':[i/10 for i in range(0,50)]}


xgb_params = {'n_estimators':[5,10,25,50,100,200,300,500]}

xgb1 = XGBRegressor(objective = 'count:poisson',
                    eta = 0.05,
                    n_estimators = 75,
                    min_child_weight = 8,
                    max_depth = 8,
                    gamma =3,
                    subsample= 0.7,
                    colsample_bytree= 1,
                    colsample_bylevel=0.9,
                    reg_lambda=2.3, 
                    reg_alpha=3.4,
                    eval_metric = 'poisson-nloglik',
                    seed = 0
                   )

random_search = RandomizedSearchCV(xgb1,
                                  cv=10,
                                  param_distributions = xgb_params,
                                  scoring ='neg_mean_absolute_error',
                                  verbose = False,
                                  n_jobs=-1)

random_search.fit(x_train,y_train)

In [ ]:
random_search.best_params_

In [ ]:
preds = random_search.predict(x_val)

from sklearn.metrics import mean_absolute_error as mae

score = 1-(mae(y_val,preds)/5.4)

score

In [ ]:
f6i

In [ ]:
sdf

In [ ]:
from sklearn.linear_model import Lasso

# you should try other algos for improvement in performance 
# the target here is not really suitable for linear regression or regression cost in general
# as it takes always positive integer values 
# you should try PoissonRegressor available with sklearn .23 and onwards 
# if that is not available , you can also make use of objective :'count:poisson' with xgboost
# you can also try HistGradientBoostingRegressor with loss='poisson'


In [ ]:
params={'alpha':np.linspace(0.01,10,100)}

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
model=Lasso(fit_intercept=True)

In [ ]:
grid_search=GridSearchCV(model,cv=10,scoring='neg_mean_absolute_error',
                         param_grid=params,n_jobs=-1,verbose=10)

# you can also make use of mean_poisson_deviance as scoring , this ignores non-positive outcomes
# it might not be avalaible with your version

In [ ]:
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
# you can see that best estimator is at the edge of the alpha range that we tried . You
# can expand on that, if you are using Lasso

In [ ]:
abs(grid_search.best_score_)

In [ ]:
submissions=pd.DataFrame({'Hazard':grid_search.predict(x_test)})

In [ ]:
submissions.to_csv('sample_submission.csv',index=False)